https://meet.google.com/hzz-msjq-jtw

# api 크롤링

기존의 크롤링 방식은
1. 내가 직접 홈페이지에 브라우저를 켜서 접근하거나(selenium)<br>

혹은<br>

2. 파이썬 자체적으로 호출을 요청해서(requests)<br>
데이터를 가지고 오는 방식이었습니다.
이 방식은 서버에 부하가 많이 갈 뿐더러 나에게 필요없는 데이터까지 한 번에 호출해서 가져오는 문제가 있습니다.<br><br>
따라서 서비스 제공자측에서는 서버 부하를 줄이고, 사용자에게 맞춤형을 데이터를 제공하기 위해 api서버를 운영합니다.<br>
api서버는 인가된 데이터만을 개발자에게 넘겨서 서버도 안정적으로 유지하며 필요없는 동영상자료나 그림자료를 호출하지 않으므로 트래픽을 줄일 수 있습니다.<br>
(횟수 제한이 있는 사이트도 많습니다. 라이엇 데이터는 1초에 5회, 2분에 100회로 제한됩니다.)<br><br>
api 서버 접근시 보통 urllib.request 를 이용하게 됩니다.

In [1]:
# 사이트에 자료 요청
import urllib.request

# json 데이터 핸들링
import json

# DataFrame 자료형 활용
import pandas as pd

# json 데이터를 pandas DataFrame으로 변환
from pandas.io.json import json_normalize

## 영진위 api 신청

https://www.kobis.or.kr/kobisopenapi/homepg/apiservice/searchServiceInfo.do

접속 후 회원가입
- 홈페이지는 블로그 등을 기입하세요.
- 가입 후 위 사이트에서 호출요청 양식을 확인해주시면 됩니다.

In [83]:
# key 값을 살펴보니 날짜에 대한 value값이 존재했고 일일이 넣기 귀찮으니 펑션을 써서 넣어야겠음
# 단, 변수 지정이 필요하므로 위 source 보다 먼저 실행해야함
from datetime import datetime, timedelta  # 현재 날짜를 가져오고, 계산해주는 라이브러리
days_tmp = 3                              # 현재로부터 며칠이 차감될지 정해지는 변수값
nowdt = datetime.now()                    # 오늘 날짜 변수(nowdt)에 입력
tmpdt = nowdt - timedelta(days=days_tmp)  # 오늘 날짜에서 날 기준으로 일정일 차감
target_dt_new = tmpdt.strftime('%Y%m%d')  # 날짜 포맷을 'yyyymmdd'형식으로 변경해 변수에 지정

In [73]:
key = '0f8d78e337d8aa7f3b6a6f192bf15749'
target_dt = '20221025'
url = f'http://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key={key}&targetDt={target_dt_new}'
url

'http://kobis.or.kr/kobisopenapi/webservice/rest/boxoffice/searchDailyBoxOfficeList.json?key=0f8d78e337d8aa7f3b6a6f192bf15749&targetDt=20221023'

### json데이터를 팬더스 데이터프레임으로 변환
api 데이터는 보통 json(JavaScript Object Notation) 타입으로 받아집니다.<br>
쉽게 말하면 자바스크립트 데이터를 전달하기 좋게 설정한 자료형인데 파이썬의 딕셔너리와 거의 같다고 보시면 됩니다.<br>
따라서 json 데이터를 팬더스 데이터로 변환한다는 것은 사실상 딕셔너리 데이터를 팬더스 데이터프레임으로 변환하는것입니다.

In [4]:
# 위에 선언해둔 url을 이용해 api 데이터 파이썬 내부로 가져오기
result = urllib.request.urlopen(url)

In [5]:
# 요청 페이지의 결과 데이터를 파이썬 변수에 저장
json_raw_data = result.read()

In [11]:
# .read()는 단 한번만 데이터를 출력해주고, 두 번째부터는 데이터가 모두 증발해버림
# 때문에 반드시 변수에 저장해줘야 함
result.read()

b''

In [10]:
# b'~~~~'로 시작하는 이 데이터는 파이썬 내부적으로 byte 자료형으로 처리되며 데이터프레임으로 바로 변환할 수 없음.
print(json_raw_data)

# byte 자료형임을 확인
type(json_raw_data)

b'{"boxOfficeResult":{"boxofficeType":"\xec\x9d\xbc\xeb\xb3\x84 \xeb\xb0\x95\xec\x8a\xa4\xec\x98\xa4\xed\x94\xbc\xec\x8a\xa4","showRange":"20221025~20221025","dailyBoxOfficeList":[{"rnum":"1","rank":"1","rankInten":"0","rankOldAndNew":"OLD","movieCd":"20226886","movieNm":"\xeb\xb8\x94\xeb\x9e\x99 \xec\x95\x84\xeb\x8b\xb4","openDt":"2022-10-19","salesAmt":"267103608","salesShare":"34.3","salesInten":"-32102353","salesChange":"-10.7","salesAcc":"5058107191","audiCnt":"26562","audiInten":"-3154","audiChange":"-10.6","audiAcc":"484264","scrnCnt":"1360","showCnt":"5143"},{"rnum":"2","rank":"2","rankInten":"0","rankOldAndNew":"OLD","movieCd":"20198317","movieNm":"\xec\x9d\xb8\xec\x83\x9d\xec\x9d\x80 \xec\x95\x84\xeb\xa6\x84\xeb\x8b\xa4\xec\x9b\x8c","openDt":"2022-09-28","salesAmt":"116727012","salesShare":"15.0","salesInten":"5077977","salesChange":"4.5","salesAcc":"9460206546","audiCnt":"13261","audiInten":"856","audiChange":"6.9","audiAcc":"1006562","scrnCnt":"710","showCnt":"2004"},{"rnum

bytes

In [14]:
# byte 자료형은 한글이 깨져서 나오므로 .decode('utf-8') 을 이용해 utf-8로 고쳐줘야 함
# encode => 우리가 사용하는 문자를 컴퓨터언어로 변환
# decode => 컴퓨터언어를 우리가 사용하는 문자로 변환
json_utf8 = json_raw_data.decode('utf-8')

# decode 후 b'~~~~' 에서 b 가 빠지고 깨졌던 한글이 재구성됨
print(json_utf8)

# byte 에서 str 로 형변환됨
type(json_utf8)

{"boxOfficeResult":{"boxofficeType":"일별 박스오피스","showRange":"20221025~20221025","dailyBoxOfficeList":[{"rnum":"1","rank":"1","rankInten":"0","rankOldAndNew":"OLD","movieCd":"20226886","movieNm":"블랙 아담","openDt":"2022-10-19","salesAmt":"267103608","salesShare":"34.3","salesInten":"-32102353","salesChange":"-10.7","salesAcc":"5058107191","audiCnt":"26562","audiInten":"-3154","audiChange":"-10.6","audiAcc":"484264","scrnCnt":"1360","showCnt":"5143"},{"rnum":"2","rank":"2","rankInten":"0","rankOldAndNew":"OLD","movieCd":"20198317","movieNm":"인생은 아름다워","openDt":"2022-09-28","salesAmt":"116727012","salesShare":"15.0","salesInten":"5077977","salesChange":"4.5","salesAcc":"9460206546","audiCnt":"13261","audiInten":"856","audiChange":"6.9","audiAcc":"1006562","scrnCnt":"710","showCnt":"2004"},{"rnum":"3","rank":"3","rankInten":"0","rankOldAndNew":"OLD","movieCd":"20215601","movieNm":"공조2: 인터내셔날","openDt":"2022-09-07","salesAmt":"79190686","salesShare":"10.2","salesInten":"1690605","salesChange":

str

In [15]:
# str 을 dict 로 변환하기 위해 json.loads(자료)를 활용
# 딕셔너리 형태의 문자열을 딕셔너리로 변환
json_complete = json.loads(json_utf8)
print(json_complete)

# 딕셔너리로 자료형 변환 완료
type(json_complete)

{'boxOfficeResult': {'boxofficeType': '일별 박스오피스', 'showRange': '20221025~20221025', 'dailyBoxOfficeList': [{'rnum': '1', 'rank': '1', 'rankInten': '0', 'rankOldAndNew': 'OLD', 'movieCd': '20226886', 'movieNm': '블랙 아담', 'openDt': '2022-10-19', 'salesAmt': '267103608', 'salesShare': '34.3', 'salesInten': '-32102353', 'salesChange': '-10.7', 'salesAcc': '5058107191', 'audiCnt': '26562', 'audiInten': '-3154', 'audiChange': '-10.6', 'audiAcc': '484264', 'scrnCnt': '1360', 'showCnt': '5143'}, {'rnum': '2', 'rank': '2', 'rankInten': '0', 'rankOldAndNew': 'OLD', 'movieCd': '20198317', 'movieNm': '인생은 아름다워', 'openDt': '2022-09-28', 'salesAmt': '116727012', 'salesShare': '15.0', 'salesInten': '5077977', 'salesChange': '4.5', 'salesAcc': '9460206546', 'audiCnt': '13261', 'audiInten': '856', 'audiChange': '6.9', 'audiAcc': '1006562', 'scrnCnt': '710', 'showCnt': '2004'}, {'rnum': '3', 'rank': '3', 'rankInten': '0', 'rankOldAndNew': 'OLD', 'movieCd': '20215601', 'movieNm': '공조2: 인터내셔날', 'openDt': '

dict

In [41]:
# 1차적으로 먼저 pandas 데이터프레임화
# dailyBoxOfficeList에 실제 원하는 데이터가 들어가있음
pd.DataFrame(json_complete)

# json_compolete 계층은 먼저 boxOfficeResult를 가져와야 하위 접근이 가능함
pd.DataFrame(json_complete['boxOfficeResult'])

# 원하는 데이터가 맞는지 확인  => 리스트화 되어있음
print(json_complete['boxOfficeResult']['dailyBoxOfficeList'])

# 최종적으로 boxOfficeResult 하위 데이터 접근 및 DataFrame화 해서 변수에 저장
daily_data = pd.DataFrame(json_complete['boxOfficeResult']['dailyBoxOfficeList'])
daily_data

[{'rnum': '1', 'rank': '1', 'rankInten': '0', 'rankOldAndNew': 'OLD', 'movieCd': '20226886', 'movieNm': '블랙 아담', 'openDt': '2022-10-19', 'salesAmt': '267103608', 'salesShare': '34.3', 'salesInten': '-32102353', 'salesChange': '-10.7', 'salesAcc': '5058107191', 'audiCnt': '26562', 'audiInten': '-3154', 'audiChange': '-10.6', 'audiAcc': '484264', 'scrnCnt': '1360', 'showCnt': '5143'}, {'rnum': '2', 'rank': '2', 'rankInten': '0', 'rankOldAndNew': 'OLD', 'movieCd': '20198317', 'movieNm': '인생은 아름다워', 'openDt': '2022-09-28', 'salesAmt': '116727012', 'salesShare': '15.0', 'salesInten': '5077977', 'salesChange': '4.5', 'salesAcc': '9460206546', 'audiCnt': '13261', 'audiInten': '856', 'audiChange': '6.9', 'audiAcc': '1006562', 'scrnCnt': '710', 'showCnt': '2004'}, {'rnum': '3', 'rank': '3', 'rankInten': '0', 'rankOldAndNew': 'OLD', 'movieCd': '20215601', 'movieNm': '공조2: 인터내셔날', 'openDt': '2022-09-07', 'salesAmt': '79190686', 'salesShare': '10.2', 'salesInten': '1690605', 'salesChange': '2.2', 

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt
0,1,1,0,OLD,20226886,블랙 아담,2022-10-19,267103608,34.3,-32102353,-10.7,5058107191,26562,-3154,-10.6,484264,1360,5143
1,2,2,0,OLD,20198317,인생은 아름다워,2022-09-28,116727012,15.0,5077977,4.5,9460206546,13261,856,6.9,1006562,710,2004
2,3,3,0,OLD,20215601,공조2: 인터내셔날,2022-09-07,79190686,10.2,1690605,2.2,70423477274,9634,189,2,6913610,599,1597
3,4,4,0,OLD,20226798,에브리씽 에브리웨어 올 앳 원스,2022-10-12,60339012,7.7,2315770,4,1339037716,6020,322,5.7,126221,406,691
4,5,5,0,OLD,20226777,극장판 짱구는 못말려: 수수께끼! 꽃피는 천하떡잎학교,2022-09-28,40984965,5.3,1815783,4.6,6477183941,4169,256,6.5,646160,423,623
5,6,6,2,OLD,20227225,귀못,2022-10-19,26733800,3.4,6588200,32.7,290924700,4022,1430,55.2,34386,202,447
6,7,7,-1,OLD,20227304,오펀: 천사의 탄생,2022-10-12,32400877,4.2,-4134308,-11.3,1662085280,3572,-219,-5.8,161590,417,653
7,8,8,-1,OLD,20219628,정직한 후보2,2022-09-28,26879094,3.5,3455603,14.8,8339439168,3221,454,16.4,885033,377,544
8,9,9,1,OLD,20225729,스마일,2022-10-06,19397394,2.5,9083798,88.1,1068164072,1956,1014,107.6,100338,87,122
9,10,10,1,OLD,20226877,미혹,2022-10-19,11844100,1.5,3582501,43.4,94829791,1457,585,67.1,11560,164,237


In [44]:
daily_data.columns

Index(['rnum', 'rank', 'rankInten', 'rankOldAndNew', 'movieCd', 'movieNm',
       'openDt', 'salesAmt', 'salesShare', 'salesInten', 'salesChange',
       'salesAcc', 'audiCnt', 'audiInten', 'audiChange', 'audiAcc', 'scrnCnt',
       'showCnt'],
      dtype='object')

In [45]:
# 얻어온 데이터는 빈 pandas DataFrame을 만들고, 거기에 날짜 컬럼을 추가하면서 concat으로 누적시키면 됨
# 컬럼값은 daily_data에 있는 컬럼값을 긁어와 넣는다
final_data = pd.DataFrame(columns=daily_data.columns)
final_data

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt


In [48]:
# 기존 최종 데이터 + 오늘 날짜로 조회된 새로운 데이터
final_data = pd.concat([final_data, daily_data])

In [49]:
final_data

,rnum,rank,rankInten,rankOldAndNew,movieCd,movieNm,openDt,salesAmt,salesShare,salesInten,salesChange,salesAcc,audiCnt,audiInten,audiChange,audiAcc,scrnCnt,showCnt
0,1,1,0,OLD,20226886,블랙 아담,2022-10-19,267103608,34.3,-32102353,-10.7,5058107191,26562,-3154,-10.6,484264,1360,5143
1,2,2,0,OLD,20198317,인생은 아름다워,2022-09-28,116727012,15.0,5077977,4.5,9460206546,13261,856,6.9,1006562,710,2004
2,3,3,0,OLD,20215601,공조2: 인터내셔날,2022-09-07,79190686,10.2,1690605,2.2,70423477274,9634,189,2,6913610,599,1597
3,4,4,0,OLD,20226798,에브리씽 에브리웨어 올 앳 원스,2022-10-12,60339012,7.7,2315770,4,1339037716,6020,322,5.7,126221,406,691
4,5,5,0,OLD,20226777,극장판 짱구는 못말려: 수수께끼! 꽃피는 천하떡잎학교,2022-09-28,40984965,5.3,1815783,4.6,6477183941,4169,256,6.5,646160,423,623
5,6,6,2,OLD,20227225,귀못,2022-10-19,26733800,3.4,6588200,32.7,290924700,4022,1430,55.2,34386,202,447
6,7,7,-1,OLD,20227304,오펀: 천사의 탄생,2022-10-12,32400877,4.2,-4134308,-11.3,1662085280,3572,-219,-5.8,161590,417,653
7,8,8,-1,OLD,20219628,정직한 후보2,2022-09-28,26879094,3.5,3455603,14.8,8339439168,3221,454,16.4,885033,377,544
8,9,9,1,OLD,20225729,스마일,2022-10-06,19397394,2.5,9083798,88.1,1068164072,1956,1014,107.6,100338,87,122
9,10,10,1,OLD,20226877,미혹,2022-10-19,11844100,1.5,3582501,43.4,94829791,1457,585,67.1,11560,164,237


# 날짜 처리

영진위 데이터는 날짜를 yyyymmdd 형식으로(ex 2022년 10월 26일 -> 20221026) 처리합니다.

문제는 그냥 문자열로 넘기면 20221231 + 1 이 20220101 이 아닌 20221232  가 된다

날짜타입을 따로 처리해줘야 월이나 년 증가, 그리고 윤년까지 처리할 수 있으며 날짜를 편하게 반복문으로 입력 가능

In [74]:
from datetime import date, timedelta

In [75]:
today = date(2022, 10, 26)
today

datetime.date(2022, 10, 26)

In [76]:
# datetime 날짜에 특정 연, 월, 일 만큼 증감하고 싶다면
# timedelta(years=연, months=월, weeks=주, days=일)
today + timedelta(days=100)

datetime.date(2023, 2, 3)

In [82]:
# 날짜를 문자열로 변환
# %y : 2000 번대를 뺀 연도, %Y : 2000 번대를 포함한 연도
# %m : 월
# %d : 일,   %D : 날짜를 YY/MM/DD 형식으로 출력
today.strftime('%m %M %d %D')

'10 00 26 10/26/22'